In [1]:
# Library
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import DictionaryLearning, sparse_encode
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.decomposition import SparseCoder

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

# Load data
measurement = np.load('../test/dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:1000]  # Using 1000 samples for simplicity

# Split data
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)

# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6    # Channel 6 is EVE (illegitimate)

# Set the number of dictionary atoms (components) and sparsity level
# N_components = 3  # Number of Sparse Components
n_components = 10  # Try values like 20, 50, or 100
n_nonzero_coefs = 5 # Sparsity level

In [2]:
# ----------------------------------------------------- Preprocessing -----------------------------------------------------
# ----------------- Training data -----------------
train_alice_cirs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
train_eve_cirs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs
train_alice_magnitude = np.abs(train_alice_cirs[..., 0] + 1j * train_alice_cirs[..., 1])
train_eve_magnitude = np.abs(train_eve_cirs[..., 0] + 1j * train_eve_cirs[..., 1])
train_data_combined  = np.vstack((train_alice_magnitude, train_eve_magnitude))

train_dict = train_data_combined.T

traincoder = SparseCoder(dictionary=train_dict, transform_algorithm='omp', transform_n_nonzero_coefs=n_nonzero_coefs)
train_sparse_codes = traincoder.transform(train_data_combined)

print(train_sparse_codes.shape)
print(train_sparse_codes[0])

ValueError: X has 251 features, but SparseCoder is expecting 1600 features as input.

In [9]:
test_alice_CIRs = testCIR[:, alice_channel, :, :]
test_eve_CIRs = testCIR[:, eve_channel, :, :]

# test amplitude
test_alice_magnitude = np.abs(test_alice_CIRs[..., 0] + 1j * test_alice_CIRs[..., 1])
test_eve_magnitude = np.abs(test_eve_CIRs[..., 0] + 1j * test_eve_CIRs[..., 1]) 
test_data_combined = np.vstack((test_alice_magnitude, test_eve_magnitude))

test_dict = test_data_combined.T

testcoder = SparseCoder(dictionary=test_dict, transform_algorithm='omp', transform_n_nonzero_coefs=n_nonzero_coefs)
test_sparse_codes = testcoder.transform(test_data_combined.T)
print('Test data shape:', test_sparse_codes.shape)


Test data shape: (251, 251)


/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/utils/_param_validation.py:186: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  return func(*args, **kwargs)


In [10]:
# Labels for training data
train_alice_labels = np.zeros(train_alice_sparse_codes.shape[0])  # Label '0' for Alice.
train_eve_labels = np.ones(train_eve_sparse_codes.shape[0])       # Label '1' for Eve.
train_labels = np.hstack((train_alice_labels, train_eve_labels))

# Labels for test data
test_alice_labels = np.zeros(test_alice_sparse_codes.shape[0])  # Label '0' for Alice.
test_eve_labels = np.ones(test_eve_sparse_codes.shape[0])       # Label '1' for Eve.
test_labels = np.hstack((test_alice_labels, test_eve_labels))

NameError: name 'train_alice_sparse_codes' is not defined

In [ ]:
scaler = StandardScaler()
train_dict= scaler.fit_transform(train_sparse_codes)
# Apply the same scaling to test data
test_sparse_codes = scaler.transform(test_sparse_codes)
print('Train data shape:', train_sparse_codes.shape)